# Procesamiento en lote de imágenes satelitales para segmentación urbano-rural

Este notebook procesa todas las imágenes satelitales de Querétaro (1984-2020) en la carpeta `data/raw` y genera mapas de segmentación urbano-rural-vegetación que se guardan en `data/processed`.

## Proceso de transformación:
1. **Extracción de características**: LBP, Sobel, HSV, varianza local
2. **Clustering K-means**: Agrupación de píxeles similares  
3. **Asignación semántica**: Clasificación en clases urbanas/rurales
4. **Refinamiento espacial**: Operaciones morfológicas y filtrado
5. **Simplificación**: Agrupación en 3 categorías (Urbano, Vegetación, Rural)

In [1]:
# 1. Importar librerías necesarias
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import cv2
from PIL import Image
import pickle
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Librerías para procesamiento de imágenes
from skimage.feature import local_binary_pattern
from skimage import filters, morphology, measure, segmentation
from scipy import ndimage
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score

print("✅ Librerías importadas correctamente")
print(f"Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Librerías importadas correctamente
Fecha de ejecución: 2025-10-12 13:26:20


In [2]:
# 2. Configurar rutas y verificar archivos
project_root = Path(r"D:\maestria\Tesis\AC-CityExpansion")
raw_data_path = project_root / "data" / "raw"
processed_data_path = project_root / "data" / "processed" 
results_path = project_root / "results"

# Crear directorios si no existen
processed_data_path.mkdir(parents=True, exist_ok=True)
results_path.mkdir(parents=True, exist_ok=True)

# Buscar todas las imágenes en la carpeta raw
image_files = list(raw_data_path.glob("*.jpg"))
image_files.extend(list(raw_data_path.glob("*.png")))
image_files.extend(list(raw_data_path.glob("*.tif")))

# Filtrar archivos que no son metadatos
image_files = [f for f in image_files if not f.name.startswith('.') and 'metadata' not in f.name.lower()]
image_files.sort()  # Ordenar por nombre (cronológicamente)

print(f"📁 Rutas configuradas:")
print(f"  • Datos originales: {raw_data_path}")
print(f"  • Datos procesados: {processed_data_path}")
print(f"  • Resultados: {results_path}")
print(f"\n📸 Imágenes encontradas: {len(image_files)}")

# Mostrar las primeras y últimas imágenes
if image_files:
    print(f"  • Primera imagen: {image_files[0].name}")
    print(f"  • Última imagen: {image_files[-1].name}")
    
    # Extraer años de los nombres de archivo
    years = []
    for img_file in image_files:
        # Extraer año del nombre del archivo (formato: qrtro_12_YYYY.jpg)
        try:
            year = int(img_file.stem.split('_')[-1])
            years.append(year)
        except:
            print(f"⚠️ No se pudo extraer el año de: {img_file.name}")
    
    if years:
        print(f"  • Rango temporal: {min(years)} - {max(years)} ({len(set(years))} años únicos)")
else:
    print("❌ No se encontraron imágenes para procesar")

📁 Rutas configuradas:
  • Datos originales: D:\maestria\Tesis\AC-CityExpansion\data\raw
  • Datos procesados: D:\maestria\Tesis\AC-CityExpansion\data\processed
  • Resultados: D:\maestria\Tesis\AC-CityExpansion\results

📸 Imágenes encontradas: 37
  • Primera imagen: qrtro_12_1984.jpg
  • Última imagen: qrtro_12_2020.jpg
  • Rango temporal: 1984 - 2020 (37 años únicos)


In [3]:
# 3. Funciones para extracción de características (basadas en el notebook original)

def extract_lbp_features(image, radius=3, n_points=24):
    """Extrae características Local Binary Pattern"""
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) if len(image.shape) == 3 else image
    lbp = local_binary_pattern(gray, n_points, radius, method='uniform')
    return lbp.astype(np.float32)

def extract_sobel_features(image):
    """Extrae características de gradientes Sobel"""
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) if len(image.shape) == 3 else image
    sobel_x = filters.sobel_h(gray)
    sobel_y = filters.sobel_v(gray) 
    sobel_mag = np.sqrt(sobel_x**2 + sobel_y**2)
    return sobel_mag.astype(np.float32)

def extract_color_features(image):
    """Extrae características de color HSV"""
    if len(image.shape) == 3:
        hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        return hsv[:,:,1].astype(np.float32)  # Saturación
    else:
        return np.zeros_like(image, dtype=np.float32)

def extract_variance_features(image, window_size=15):
    """Extrae características de varianza local"""
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) if len(image.shape) == 3 else image
    
    # Calcular varianza local usando filtros
    kernel = np.ones((window_size, window_size)) / (window_size * window_size)
    mean = cv2.filter2D(gray.astype(np.float32), -1, kernel)
    mean_sq = cv2.filter2D((gray.astype(np.float32))**2, -1, kernel)
    variance = mean_sq - mean**2
    
    return variance.astype(np.float32)

def extract_all_features(image):
    """Extrae todas las características de una imagen"""
    print("    📊 Extrayendo características...")
    
    # LBP
    lbp = extract_lbp_features(image)
    
    # Sobel
    sobel = extract_sobel_features(image)
    
    # Color (saturación HSV)
    saturation = extract_color_features(image)
    
    # Varianza local
    variance = extract_variance_features(image)
    
    # Combinar características en un array plano
    h, w = image.shape[:2]
    features = np.column_stack([
        lbp.flatten(),
        sobel.flatten(), 
        saturation.flatten(),
        variance.flatten()
    ])
    
    return features, (h, w)

print("✅ Funciones de extracción de características definidas")

✅ Funciones de extracción de características definidas


In [ ]:
# 4. Funciones para clustering y clasificación semántica (CORREGIDAS)

def perform_clustering(features, n_clusters=5, random_state=42):
    """Realiza clustering K-means en las características"""
    print(f"    🎯 Aplicando K-means con {n_clusters} clusters...")
    
    # Normalizar características
    scaler = RobustScaler()
    features_scaled = scaler.fit_transform(features)
    
    # Aplicar K-means
    kmeans = MiniBatchKMeans(
        n_clusters=n_clusters, 
        random_state=random_state,
        batch_size=1000,
        max_iter=100
    )
    labels = kmeans.fit_predict(features_scaled)
    
    return labels, kmeans, scaler

def classify_clusters_to_semantic(features, labels, image_shape):
    """Clasifica clusters en categorías semánticas urbano-rural-vegetación (MEJORADA)"""
    print("    🏷️ Asignando clases semánticas...")
    
    # Calcular estadísticas por cluster
    unique_labels = np.unique(labels)
    cluster_stats = {}
    
    h, w = image_shape
    labels_2d = labels.reshape(h, w)
    
    for cluster_id in unique_labels:
        mask = labels_2d == cluster_id
        if np.sum(mask) == 0:
            continue
            
        # Estadísticas de características
        cluster_features = features[labels == cluster_id]
        
        lbp_mean = np.mean(cluster_features[:, 0])
        sobel_mean = np.mean(cluster_features[:, 1])
        sat_mean = np.mean(cluster_features[:, 2])
        var_mean = np.mean(cluster_features[:, 3])
        
        cluster_stats[cluster_id] = {
            'lbp': lbp_mean,
            'sobel': sobel_mean,
            'saturation': sat_mean,
            'variance': var_mean,
            'size': np.sum(mask)
        }
    
    # REGLAS DE CLASIFICACIÓN MEJORADAS Y MÁS ROBUSTAS
    semantic_mapping = {}
    
    # Normalizar estadísticas para mejor comparación
    all_sobel = [stats['sobel'] for stats in cluster_stats.values()]
    all_sat = [stats['saturation'] for stats in cluster_stats.values()]
    all_var = [stats['variance'] for stats in cluster_stats.values()]
    all_lbp = [stats['lbp'] for stats in cluster_stats.values()]
    
    # Calcular percentiles para clasificación adaptativa
    sobel_high = np.percentile(all_sobel, 70)
    sobel_low = np.percentile(all_sobel, 30)
    sat_high = np.percentile(all_sat, 70)
    sat_low = np.percentile(all_sat, 30)
    var_high = np.percentile(all_var, 70)
    lbp_high = np.percentile(all_lbp, 70)
    
    print(f"    📊 Umbrales adaptativos:")
    print(f"      • Sobel alto: {sobel_high:.3f}, bajo: {sobel_low:.3f}")
    print(f"      • Saturación alta: {sat_high:.3f}, baja: {sat_low:.3f}")
    print(f"      • Varianza alta: {var_high:.3f}")
    
    for cluster_id, stats in cluster_stats.items():
        size_factor = stats['size'] / (h * w)  # Proporción del cluster
        
        # REGLAS DE CLASIFICACIÓN MEJORADAS:
        
        # 1. URBANO: Alto contraste (Sobel) + textura compleja (LBP alto)
        if (stats['sobel'] > sobel_high and stats['lbp'] > lbp_high) or \
           (stats['sobel'] > sobel_high and stats['saturation'] < sat_low):
            semantic_mapping[cluster_id] = {
                'label': 'Urbano',
                'confidence': min(0.9, 0.6 + (stats['sobel'] - sobel_high) / sobel_high)
            }
            
        # 2. VEGETACIÓN: Alta saturación + varianza media-alta
        elif stats['saturation'] > sat_high and stats['variance'] > var_high:
            semantic_mapping[cluster_id] = {
                'label': 'Vegetación', 
                'confidence': min(0.9, 0.6 + (stats['saturation'] - sat_high) / sat_high)
            }
            
        # 3. CARRETERAS/INFRAESTRUCTURA: Sobel medio-alto + saturación muy baja + textura uniforme
        elif stats['sobel'] > sobel_low and stats['saturation'] < sat_low and stats['variance'] < var_high:
            semantic_mapping[cluster_id] = {
                'label': 'Urbano',  # Agrupar con urbano
                'confidence': 0.7
            }
            
        # 4. RURAL/SUELO DESNUDO: Baja saturación + baja varianza + bajo sobel
        else:
            semantic_mapping[cluster_id] = {
                'label': 'Rural',
                'confidence': 0.7
            }
    
    # VERIFICACIÓN DE DISTRIBUCIÓN BALANCEADA
    class_counts = {'Urbano': 0, 'Vegetación': 0, 'Rural': 0}
    for cluster_id, info in semantic_mapping.items():
        class_counts[info['label']] += cluster_stats[cluster_id]['size']
    
    total_pixels = sum(class_counts.values())
    for class_name, count in class_counts.items():
        percentage = 100.0 * count / total_pixels if total_pixels > 0 else 0
        print(f"    • {class_name}: {percentage:.1f}%")
    
    # Si una clase domina demasiado (>90%), redistribuir
    dominant_class = max(class_counts.items(), key=lambda x: x[1])
    if dominant_class[1] / total_pixels > 0.9:
        print(f"    ⚠️ Redistribuyendo - {dominant_class[0]} domina con {100*dominant_class[1]/total_pixels:.1f}%")
        
        # Reasignar algunos clusters grandes a otras clases
        sorted_clusters = sorted(cluster_stats.items(), key=lambda x: x[1]['size'], reverse=True)
        reassigned = 0
        
        for cluster_id, stats in sorted_clusters:
            if semantic_mapping[cluster_id]['label'] == dominant_class[0] and reassigned < 2:
                # Reasignar basándose en características secundarias
                if stats['saturation'] > np.median(all_sat):
                    semantic_mapping[cluster_id]['label'] = 'Vegetación'
                elif stats['sobel'] > np.median(all_sobel):
                    semantic_mapping[cluster_id]['label'] = 'Urbano' 
                else:
                    semantic_mapping[cluster_id]['label'] = 'Rural'
                reassigned += 1
    
    return semantic_mapping, cluster_stats

print("✅ Funciones de clustering y clasificación semántica CORREGIDAS")

✅ Funciones de clustering y clasificación semántica definidas


In [5]:
# 5. Funciones para refinamiento espacial y post-procesamiento

def apply_morphological_operations(labels, min_size=50):
    """Aplica operaciones morfológicas para limpiar la segmentación"""
    result = labels.copy()
    unique_labels = np.unique(labels)
    
    for label in unique_labels:
        if label == 0:
            continue
            
        mask = (labels == label)
        # Eliminar objetos pequeños
        cleaned = morphology.remove_small_objects(mask, min_size=min_size)
        # Cerrado morfológico
        cleaned = ndimage.binary_closing(cleaned, structure=np.ones((3,3)))
        # Rellenar agujeros
        cleaned = ndimage.binary_fill_holes(cleaned)
        
        result[mask & ~cleaned] = 0
        
    return result

def refine_segmentation(labels, image):
    """Refina la segmentación usando información espacial"""
    # Aplicar watershed para mejorar bordes
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) if len(image.shape) == 3 else image
    edges = filters.sobel(gray)
    refined = segmentation.watershed(edges, labels, mask=np.ones_like(labels, dtype=bool))
    return refined

def simplify_to_three_classes(semantic_labels, semantic_mapping):
    """Simplifica a las 3 clases finales: Urbano, Vegetación, Rural"""
    simplified_labels = np.zeros_like(semantic_labels)
    
    # Mapeo de clases semánticas a clases simplificadas
    class_mapping = {
        'Urbano': 1,
        'Vegetación': 2, 
        'Rural': 3
    }
    
    for cluster_id, info in semantic_mapping.items():
        semantic_class = info['label']
        simplified_class = class_mapping[semantic_class]
        
        mask = semantic_labels == cluster_id
        simplified_labels[mask] = simplified_class
    
    return simplified_labels

def create_rgb_visualization(simplified_labels):
    """Crea una visualización RGB de la segmentación simplificada"""
    class_colors = {
        1: [0.8, 0.1, 0.1],  # Rojo para urbano
        2: [0.1, 0.6, 0.1],  # Verde para vegetación  
        3: [0.8, 0.7, 0.3]   # Beige para rural
    }
    
    h, w = simplified_labels.shape
    rgb_map = np.zeros((h, w, 3))
    
    for class_id, color in class_colors.items():
        mask = simplified_labels == class_id
        rgb_map[mask] = color
        
    return rgb_map

print("✅ Funciones de refinamiento y post-procesamiento definidas")

✅ Funciones de refinamiento y post-procesamiento definidas


In [6]:
# 6. Función principal de procesamiento de una imagen

def process_single_image(image_path, output_dir, save_intermediates=False):
    """Procesa una sola imagen y genera la segmentación urbano-rural-vegetación"""
    
    print(f"\n🖼️ Procesando: {image_path.name}")
    
    try:
        # 1. Cargar imagen
        image = np.array(Image.open(image_path))
        if len(image.shape) == 3 and image.shape[2] == 4:  # RGBA
            image = image[:,:,:3]  # Convertir a RGB
        
        print(f"    📐 Dimensiones: {image.shape}")
        
        # 2. Extracción de características
        features, image_shape = extract_all_features(image)
        
        # 3. Clustering
        labels, kmeans, scaler = perform_clustering(features, n_clusters=5)
        
        # 4. Clasificación semántica
        semantic_mapping, cluster_stats = classify_clusters_to_semantic(features, labels, image_shape)
        
        # 5. Reshape labels a forma 2D
        labels_2d = labels.reshape(image_shape)
        
        # 6. Refinamiento espacial
        print("    🔧 Aplicando refinamiento espacial...")
        cleaned_labels = apply_morphological_operations(labels_2d)
        refined_labels = refine_segmentation(cleaned_labels, image)
        
        # 7. Simplificación a 3 clases
        print("    🎯 Simplificando a 3 clases...")
        simplified_labels = simplify_to_three_classes(refined_labels, semantic_mapping)
        
        # 8. Crear visualización RGB
        rgb_visualization = create_rgb_visualization(simplified_labels)
        
        # 9. Calcular estadísticas
        stats = {}
        class_names = {1: 'Urbano', 2: 'Vegetación', 3: 'Rural'}
        total_pixels = simplified_labels.size
        
        for class_id, class_name in class_names.items():
            count = np.sum(simplified_labels == class_id)
            percentage = 100.0 * count / total_pixels
            stats[class_name] = {
                'pixels': int(count),
                'percentage': float(percentage)
            }
        
        # 10. Preparar nombres de archivos de salida
        base_name = image_path.stem
        
        # 11. Guardar resultados
        print("    💾 Guardando resultados...")
        
        # Guardar etiquetas simplificadas como numpy array
        labels_file = output_dir / f"{base_name}_labels.npy"
        np.save(labels_file, simplified_labels)
        
        # Guardar visualización RGB
        rgb_file = output_dir / f"{base_name}_segmentation.png"
        plt.imsave(rgb_file, rgb_visualization)
        
        # Guardar estadísticas
        stats_file = output_dir / f"{base_name}_stats.json"
        with open(stats_file, 'w') as f:
            json.dump(stats, f, indent=2)
        
        # Guardar metadatos del procesamiento
        metadata = {
            'source_image': str(image_path),
            'processing_date': datetime.now().isoformat(),
            'image_shape': image_shape,
            'n_clusters': 5,
            'semantic_mapping': {str(k): v for k, v in semantic_mapping.items()},
            'statistics': stats
        }
        
        metadata_file = output_dir / f"{base_name}_metadata.json"
        with open(metadata_file, 'w') as f:
            json.dump(metadata, f, indent=2)
        
        # Opcionalmente guardar resultados intermedios
        if save_intermediates:
            # Guardar imagen original redimensionada si es necesario
            original_file = output_dir / f"{base_name}_original.png"
            plt.imsave(original_file, image)
            
            # Guardar clusters sin procesar
            clusters_rgb = create_rgb_visualization(labels_2d)
            clusters_file = output_dir / f"{base_name}_clusters.png"
            plt.imsave(clusters_file, clusters_rgb)
        
        print(f"    ✅ Completado. Estadísticas:")
        for class_name, stat in stats.items():
            print(f"      • {class_name}: {stat['pixels']:,} píxeles ({stat['percentage']:.1f}%)")
        
        return {
            'success': True,
            'stats': stats,
            'files': {
                'labels': labels_file,
                'visualization': rgb_file,
                'stats': stats_file,
                'metadata': metadata_file
            }
        }
        
    except Exception as e:
        print(f"    ❌ Error procesando {image_path.name}: {str(e)}")
        return {
            'success': False,
            'error': str(e),
            'files': {}
        }

print("✅ Función principal de procesamiento definida")

✅ Función principal de procesamiento definida


In [7]:
# 7. Procesamiento en lote de todas las imágenes

def process_all_images():
    """Procesa todas las imágenes en la carpeta raw"""
    
    print(f"\n🚀 INICIANDO PROCESAMIENTO EN LOTE")
    print(f"Número total de imágenes: {len(image_files)}")
    print("=" * 60)
    
    # Crear subdirectorio para los resultados con timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    batch_output_dir = processed_data_path / f"batch_{timestamp}"
    batch_output_dir.mkdir(exist_ok=True)
    
    # Registros para el resumen final
    processing_log = []
    successful_count = 0
    failed_count = 0
    total_stats = {'Urbano': [], 'Vegetación': [], 'Rural': []}
    
    # Procesar cada imagen
    for i, image_file in enumerate(image_files, 1):
        print(f"\n[{i}/{len(image_files)}] ", end="")
        
        result = process_single_image(
            image_file, 
            batch_output_dir, 
            save_intermediates=(i <= 3)  # Guardar intermedios solo para las primeras 3 imágenes
        )
        
        # Registrar resultado
        log_entry = {
            'image': image_file.name,
            'index': i,
            'success': result['success'],
            'timestamp': datetime.now().isoformat()
        }
        
        if result['success']:
            successful_count += 1
            log_entry.update(result)
            
            # Acumular estadísticas para análisis temporal
            for class_name, stats in result['stats'].items():
                total_stats[class_name].append({
                    'year': int(image_file.stem.split('_')[-1]),
                    'percentage': stats['percentage'],
                    'pixels': stats['pixels']
                })
        else:
            failed_count += 1
            log_entry['error'] = result['error']
        
        processing_log.append(log_entry)
        
        # Mostrar progreso cada 10 imágenes
        if i % 10 == 0 or i == len(image_files):
            progress = 100 * i / len(image_files)
            print(f"\n📊 Progreso: {progress:.1f}% ({successful_count} exitosos, {failed_count} fallidos)")
    
    # Guardar log del procesamiento
    log_file = batch_output_dir / "processing_log.json"
    with open(log_file, 'w') as f:
        json.dump(processing_log, f, indent=2)
    
    # Crear resumen de estadísticas temporales
    temporal_stats = {}
    for class_name, records in total_stats.items():
        if records:
            df = pd.DataFrame(records).sort_values('year')
            temporal_stats[class_name] = {
                'years': df['year'].tolist(),
                'percentages': df['percentage'].tolist(),
                'mean_percentage': float(df['percentage'].mean()),
                'std_percentage': float(df['percentage'].std()),
                'trend': 'increasing' if df['percentage'].iloc[-1] > df['percentage'].iloc[0] else 'decreasing'
            }
    
    # Guardar estadísticas temporales
    temporal_file = batch_output_dir / "temporal_analysis.json"
    with open(temporal_file, 'w') as f:
        json.dump(temporal_stats, f, indent=2)
    
    # Resumen final
    print(f"\n" + "=" * 60)
    print(f"🎉 PROCESAMIENTO COMPLETADO")
    print(f"📊 Resumen:")
    print(f"  • Total de imágenes: {len(image_files)}")
    print(f"  • Procesadas exitosamente: {successful_count}")
    print(f"  • Fallos: {failed_count}")
    print(f"  • Tasa de éxito: {100*successful_count/len(image_files):.1f}%")
    print(f"📁 Resultados guardados en: {batch_output_dir}")
    print(f"\n📈 Tendencias temporales (promedio por clase):")
    
    for class_name, stats in temporal_stats.items():
        if stats:
            print(f"  • {class_name}: {stats['mean_percentage']:.1f}% ± {stats['std_percentage']:.1f}% ({stats['trend']})")
    
    return {
        'output_directory': batch_output_dir,
        'successful_count': successful_count,
        'failed_count': failed_count,
        'processing_log': processing_log,
        'temporal_stats': temporal_stats
    }

# Ejecutar el procesamiento si hay imágenes
if image_files:
    batch_results = process_all_images()
else:
    print("❌ No hay imágenes para procesar")


🚀 INICIANDO PROCESAMIENTO EN LOTE
Número total de imágenes: 37

[1/37] 
🖼️ Procesando: qrtro_12_1984.jpg
    📐 Dimensiones: (936, 1920, 3)
    📊 Extrayendo características...
    🎯 Aplicando K-means con 5 clusters...
    🏷️ Asignando clases semánticas...
    🔧 Aplicando refinamiento espacial...
    🎯 Simplificando a 3 clases...
    💾 Guardando resultados...
    ✅ Completado. Estadísticas:
      • Urbano: 0 píxeles (0.0%)
      • Vegetación: 1,797,120 píxeles (100.0%)
      • Rural: 0 píxeles (0.0%)

[2/37] 
🖼️ Procesando: qrtro_12_1985.jpg
    📐 Dimensiones: (936, 1920, 3)
    📊 Extrayendo características...
    🎯 Aplicando K-means con 5 clusters...
    🏷️ Asignando clases semánticas...
    🔧 Aplicando refinamiento espacial...
    🎯 Simplificando a 3 clases...
    💾 Guardando resultados...
    ✅ Completado. Estadísticas:
      • Urbano: 0 píxeles (0.0%)
      • Vegetación: 1,797,120 píxeles (100.0%)
      • Rural: 0 píxeles (0.0%)

[3/37] 
🖼️ Procesando: qrtro_12_1986.jpg
    📐 Dimens

TypeError: Object of type WindowsPath is not JSON serializable

In [ ]:
# 8. Generar visualización de los resultados del procesamiento en lote

def create_batch_summary_visualization():
    """Crea visualizaciones resumen del procesamiento en lote"""
    
    if 'batch_results' not in globals() or not batch_results['temporal_stats']:
        print("❌ No hay resultados de procesamiento para visualizar")
        return
    
    print(f"\n📊 Generando visualizaciones de resumen...")
    
    # Preparar datos para visualización
    temporal_data = batch_results['temporal_stats']
    
    # Crear figura con múltiples subplots
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Resumen del Procesamiento en Lote - Segmentación Urbano-Rural-Vegetación', 
                 fontsize=16, fontweight='bold')
    
    # 1. Evolución temporal de cada clase
    ax1 = axes[0, 0]
    colors = {'Urbano': 'red', 'Vegetación': 'green', 'Rural': 'orange'}
    
    for class_name, stats in temporal_data.items():
        if stats['years']:
            ax1.plot(stats['years'], stats['percentages'], 
                    marker='o', label=class_name, color=colors[class_name], linewidth=2)
    
    ax1.set_xlabel('Año')
    ax1.set_ylabel('Porcentaje de cobertura (%)')
    ax1.set_title('Evolución temporal de las coberturas')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Distribución promedio por clase
    ax2 = axes[0, 1]
    class_names = list(temporal_data.keys())
    percentages = [temporal_data[name]['mean_percentage'] for name in class_names]
    colors_pie = [colors[name] for name in class_names]
    
    wedges, texts, autotexts = ax2.pie(percentages, labels=class_names, colors=colors_pie,
                                      autopct='%1.1f%%', startangle=90)
    ax2.set_title('Distribución promedio de coberturas')
    
    # 3. Tendencias de cambio
    ax3 = axes[1, 0]
    trends_data = []
    for class_name, stats in temporal_data.items():
        if len(stats['years']) > 1:
            # Calcular cambio porcentual total
            initial = stats['percentages'][0]
            final = stats['percentages'][-1]
            change = final - initial
            trends_data.append((class_name, change))
    
    if trends_data:
        names, changes = zip(*trends_data)
        colors_bar = [colors[name] for name in names]
        bars = ax3.bar(names, changes, color=colors_bar, alpha=0.7)
        
        # Añadir valores en las barras
        for bar, change in zip(bars, changes):
            height = bar.get_height()
            ax3.text(bar.get_x() + bar.get_width()/2., height,
                    f'{change:+.1f}%', ha='center', 
                    va='bottom' if height >= 0 else 'top')
        
        ax3.axhline(y=0, color='black', linestyle='-', alpha=0.3)
        ax3.set_ylabel('Cambio porcentual (%)')
        ax3.set_title('Cambio neto en el período analizado')
        ax3.grid(True, alpha=0.3)
    
    # 4. Estadísticas del procesamiento
    ax4 = axes[1, 1]
    ax4.axis('off')
    
    # Texto con estadísticas
    stats_text = f"""
ESTADÍSTICAS DEL PROCESAMIENTO:

• Total de imágenes procesadas: {batch_results['successful_count'] + batch_results['failed_count']}
• Procesamiento exitoso: {batch_results['successful_count']}
• Fallos: {batch_results['failed_count']}
• Tasa de éxito: {100*batch_results['successful_count']/(batch_results['successful_count'] + batch_results['failed_count']):.1f}%

PERÍODO ANALIZADO:
• Desde: {min(temporal_data['Urbano']['years']) if temporal_data['Urbano']['years'] else 'N/A'}
• Hasta: {max(temporal_data['Urbano']['years']) if temporal_data['Urbano']['years'] else 'N/A'}
• Años totales: {len(set(temporal_data['Urbano']['years'])) if temporal_data['Urbano']['years'] else 0}

DIRECTORIO DE SALIDA:
{batch_results['output_directory'].name}
    """
    
    ax4.text(0.05, 0.95, stats_text, transform=ax4.transAxes, fontsize=11,
            verticalalignment='top', fontfamily='monospace',
            bbox=dict(boxstyle="round,pad=0.5", facecolor="lightgray", alpha=0.8))
    
    plt.tight_layout()
    
    # Guardar la visualización
    summary_plot_file = batch_results['output_directory'] / "batch_summary.png"
    plt.savefig(summary_plot_file, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"✅ Visualización de resumen guardada en: {summary_plot_file}")

# Generar visualización si se ejecutó el procesamiento
if 'batch_results' in globals():
    create_batch_summary_visualization()
else:
    print("ℹ️ Ejecuta primero el procesamiento en lote para generar visualizaciones")

## Resultados y archivos generados

Después de ejecutar el procesamiento en lote, se generan los siguientes archivos para cada imagen:

### Por cada imagen procesada:
- **`{año}_labels.npy`**: Array NumPy con las etiquetas de segmentación (1=Urbano, 2=Vegetación, 3=Rural)
- **`{año}_segmentation.png`**: Visualización RGB de la segmentación
- **`{año}_stats.json`**: Estadísticas de cobertura (píxeles y porcentajes por clase)
- **`{año}_metadata.json`**: Metadatos del procesamiento (parámetros, mapeo semántico, etc.)

### Archivos de resumen del lote:
- **`processing_log.json`**: Log completo del procesamiento con éxitos y fallos
- **`temporal_analysis.json`**: Análisis de tendencias temporales
- **`batch_summary.png`**: Visualización resumen con gráficos de evolución temporal

### Estructura del directorio processed:
```
data/processed/
└── batch_YYYYMMDD_HHMMSS/
    ├── qrtro_12_1984_labels.npy
    ├── qrtro_12_1984_segmentation.png
    ├── qrtro_12_1984_stats.json
    ├── qrtro_12_1984_metadata.json
    ├── ...
    ├── processing_log.json
    ├── temporal_analysis.json
    └── batch_summary.png
```

Los archivos están listos para análisis posteriores de expansión urbana y cambios de uso del suelo a lo largo del tiempo.